In [1]:
import regex, time
from pathlib import Path

import pandas as pd
import litellm

import dotenv
dotenv.load_dotenv()

from local.db_manager import DBManager
from local.llm_captures_evaluator import LLMCapturesEvaluator

In [2]:
litellm.api_url = "http://localhost:4000"

captures_path = Path('data/Reduced')
db = DBManager('data/Assessment.sqlite')

In [3]:
## Scan the capture files and identify missing IDs
#
capture_files = {}

for file in captures_path.iterdir():
    file_id = regex.search(r'ID-(\d+).', file.name).group(1)
    emotion = regex.search(r'\.([ \w]+?)@', file.name).group(1)
    capture_files[int(file_id)] = {
        'path': file.name,
        'emotion': emotion
    }

missing_ids = [i for i in range(1, max(capture_files.keys()) + 1) 
               if i not in capture_files]
print("""Total of captures: {}
      Missing IDs: {}""".format(len(capture_files), missing_ids))

df_captures = pd.DataFrame.from_dict(capture_files, orient='index')
df_captures.head(2)

Total of captures: 1010
      Missing IDs: [185, 220, 286, 388, 389, 491, 593, 626, 796, 1001]


,path,emotion
1,2025-10-19_19h03m.ID-0001.anger@M-by-openai_gp...,anger
2,2025-10-19_19h10m.ID-0002.contempt@M-by-openai...,contempt


In [4]:
df_captures.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1010 entries, 1 to 1020
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   path     1010 non-null   object
 1   emotion  1010 non-null   object
dtypes: object(2)
memory usage: 23.7+ KB


In [5]:
db.save_dataframe(df_captures, table_name='files')

In [6]:
emergency_stop = False

In [ ]:
delay_seconds = 0
starting_index, last_index = 1, 840 # Evaluate single and pair emotions.
model = "openai/gpt-4.1"

evaluator = LLMCapturesEvaluator(
    model=model,
    df_captures=df_captures,
    captures_path=captures_path
)

for idx in df_captures.index:
    if idx < starting_index: continue
    if last_index > 0 and idx > last_index: break
    evaluator.evaluate_capture(idx)
    time.sleep(delay_seconds)
    if emergency_stop:
        print("Emergency stop triggered. Halting evaluation.")
        break

print("Evaluation complete. Instances processed:", 
      len(evaluator.df_evaluations))

Evaluation complete. Instances processed: 1


In [10]:
evaluator.save_to_db(db=db, table_name='evaluations', override_rule='append')